In [ ]:
import rapidfuzz

In [51]:
### Import the Libraries
# !pip install rapidfuzz
# import rapidfuzz
from OpenSSL import crypto
import base64
import json
import numpy as np
from datetime import datetime
import datetime
from tqdm import tqdm
import string
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import warnings
from decimal import Decimal
warnings.filterwarnings("ignore")
import rsa
import random
import pandas as pd
import pyodbc
import requests

###Read Configuration File

from configparser import ConfigParser
config = ConfigParser()
config.read('Config.ini')
config_data=config['SERVER']

print(config_data['read'])



DRIVER={FreeTDS};SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=FinApp@2022;TDS_Version=7.2;


In [52]:
###Class for SQL Server Database Connectivity
class  SQLConnect():
    
    conn=None
    query=None
    
    def __init__(self):
        pass
    
    
    def connect(self, dbengine):
        '''
        This method gets the data from database.  
        Parameters:
        dbengine: It is the database engine. Values for dbengine can be SQL_SERVER
        '''
        if dbengine == 'SQL_SERVER':
            self.conn = pyodbc.connect(config_data['read'])
            return self.conn
            logger.error("Error while connecting to database")

           
            
    def get_data(self, conn=None, objectname=None, query=None):
        '''
        This function loads the data from table or file to dataframe.
        
        Arguments-
            conn - connection object
            objectname - Object from which the data needs to be loaded
            query - The query string which will be fired on database to get the data
       
        Return Type:
            This method returns dataframe
        '''
        

        if conn!=None:
            self.query = query
            dataset = pd.read_sql(query, conn)
            return dataset

    
    def write_data(self,data, schema, tablename, mode='append', chunksize=200):
        """
        This method writes data to the database in the tablename and schema name specified.
        Parameters: 
            data - the data that needs to be written
            schema - Schema to which data needs to be written
            tablename - Table which needs to be loaded.
            mode - The mode in which the value needs to be added in the tablename.
                    Possible values are:
                        append- Inserts the data in the exising tables
                        replace - Drops and creates the table
                        update - If data exist, it will update, else it will insert data in the table.
        """
        #quoted = urllib.parse.quote_plus('DRIVER=FreeTDS;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
        #quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=localhost\SQLEXPRESS01;DATABASE=FINCORE_DB")
        #engine = sqlalchemy .create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
        #quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=172.16.33.27;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123")
        config_data=config["SERVER"]
        #print(config_data["connection"])
        quoted = urllib.parse.quote_plus(config_data["connection"])
        engine = sqlalchemy .create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
        conn = engine.connect()
        data.to_sql(tablename, schema=schema, conn = engine, chunksize=chunksize, index=False, if_exists=mode)
        conn.close()

In [53]:
###Step 3: Generate AES Key and encrypt/Decrypt Function



API_key =b'87vo4l2pp5mdsi71cz2wbnpz8wd6xa4a'

iv=bytearray(16)


def encrypt_AES_CBC(Key,data):
    data = pad(data, AES.block_size)
    cipher = AES.new(Key, AES.MODE_CBC ,iv=iv)
    encrypted_text = cipher.encrypt(data)
    return encrypted_text


In [54]:
### Step 3: Decrypt for Testing
def decrypt_AES_CBC (Key ,encrypted_text):
    cipher = AES.new(Key, AES.MODE_CBC , iv=iv)
    decrypted_text = unpad(cipher.decrypt(encrypted_text), AES.block_size)
    return decrypted_text


In [55]:
### Import CDSL Request Data to DataFrame
SQLConn  = SQLConnect()
conn = SQLConn.connect('SQL_SERVER')
cursor = conn.cursor()
status_code=[]
output = []
# new_dataframe=pd.DataFrame()

REQ_Query='''SELECT [Seqid]
      ,coalesce([Pan],'') as Pan
      ,coalesce([Account_Number],'') as [Account_Number]
      ,coalesce([Ifsc_Code],'') as [Ifsc_Code]
  FROM [EXTL_DB].[CDSL].[Cdsl_Request_Input] where [Api_Request_Sent] IS NULL'''


dataset = SQLConn.get_data(conn=conn,query=REQ_Query)


for i,row in tqdm(dataset.iterrows()):
    #print(row['Pan'])
    text = {"pan":row['Pan'],"ifsc":row['Ifsc_Code'],"bankaccountno":row["Account_Number"]}
    text=bytes(json.dumps(text,separators=(',', ':')),'ASCII')
#     print(text)
    encrypted_text=encrypt_AES_CBC(API_key,text)
#     print(encrypted_text)
    EncryptedPayload=base64.b64encode(encrypted_text).decode('ascii')
    Encrypted_body = json.dumps({'acctdtls':EncryptedPayload},separators=(',', ':'))
#     print(Encrypted_body)
    
#     print(str(row['Seqid']))
    
    url = "http://apiextappuat01.finnet2.local:8280/cdsl/1.0.0/FIUinq/getdetails"
    headers = {
        'Content-Type':'application/json',
        'seqid':str(row['Seqid']),
        'userid':'fiuuser1',
        'password':'K/Hfs9LLagBgpAuxYGQvvQ==', 
        'version':'1.0',
        'Accept':'*/*',
        'apikey':'eyJ4NXQiOiJNell4TW1Ga09HWXdNV0kwWldObU5EY3hOR1l3WW1NNFpUQTNNV0kyTkRBelpHUXpOR00wWkdSbE5qSmtPREZrWkRSaU9URmtNV0ZoTXpVMlpHVmxOZyIsImtpZCI6ImdhdGV3YXlfY2VydGlmaWNhdGVfYWxpYXMiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJmaXVkZXYwMUBjYXJib24uc3VwZXIiLCJhcHBsaWNhdGlvbiI6eyJvd25lciI6ImZpdWRldjAxIiwidGllclF1b3RhVHlwZSI6bnVsbCwidGllciI6IlVubGltaXRlZCIsIm5hbWUiOiJFeHRlcm5hbC1DRFNMLUFwcGxpY2F0aW9uIiwiaWQiOjE4MiwidXVpZCI6ImMxOTllMzRmLWQyYWItNDZjOS05ZjNmLWYyZmY0NGQ4NzBkMSJ9LCJpc3MiOiJodHRwczpcL1wvd3NvMmFtLWFtLXVhdC5hcHBzLnN1cC5maW5uZXQyLmxvY2FsOjQ0M1wvb2F1dGgyXC90b2tlbiIsInRpZXJJbmZvIjp7IlVubGltaXRlZCI6eyJ0aWVyUXVvdGFUeXBlIjoicmVxdWVzdENvdW50IiwiZ3JhcGhRTE1heENvbXBsZXhpdHkiOjAsImdyYXBoUUxNYXhEZXB0aCI6MCwic3RvcE9uUXVvdGFSZWFjaCI6dHJ1ZSwic3Bpa2VBcnJlc3RMaW1pdCI6MCwic3Bpa2VBcnJlc3RVbml0IjpudWxsfX0sImtleXR5cGUiOiJQUk9EVUNUSU9OIiwicGVybWl0dGVkUmVmZXJlciI6IiIsInN1YnNjcmliZWRBUElzIjpbeyJzdWJzY3JpYmVyVGVuYW50RG9tYWluIjoiY2FyYm9uLnN1cGVyIiwibmFtZSI6IkV4dGVybmFsLUNEU0wiLCJjb250ZXh0IjoiXC9jZHNsXC8xLjAuMCIsInB1Ymxpc2hlciI6ImZpdWRldjAxIiwidmVyc2lvbiI6IjEuMC4wIiwic3Vic2NyaXB0aW9uVGllciI6IlVubGltaXRlZCJ9XSwicGVybWl0dGVkSVAiOiIiLCJpYXQiOjE2Njg3Njk5NTMsImp0aSI6ImY5ZjdiMmJmLWQ4ZjMtNDRlZS05MzM3LTcyNmI0ZTczMDM1ZSJ9.kSGVOFGfFS3j_PzBqWZyUMqLVu2_Hfy1juySIa2sJVUemBZZSkq6qecWNxin9mWSGKIJ2Pyra2_K7HRvUh2b9rbzicpUPlQ_LN1QVrgkO7vBum7U6fst4ioZV1KzdrhOYBFWm0ZKAEp4uI6rX-ldPIB4DyqzZWZKyzx9UgskcKU6dK2l0Bf1-xKZe2ZIKpDr-oOB1u2Q_ypOu41di2PmDztY-YnZVSWmNw4tnka3bswRxe5LlJcyxlN0IuWN2tJg4rRhP6Q4TjTeoPUNiVSWtMvam0CK4wJenlIfcPOPuVIOSIAmKDgiBKx6PoDCWoyADNs4WXHn-VpOaers7Y_qZw=='
       }
    data=json.loads(Encrypted_body)
    response = requests.post(url, headers=headers , json=data)
#     print(response.status_code)
    status_code.append(response.status_code)
    Encrypted_body=response.json()
    print(Encrypted_body)
    b64 = json.loads(Encrypted_body)
#     print(b64)
    encrypted_text=base64.b64decode(b64['resAcctdtls'])
#     print(encrypted_text)
    descryptedMsg=decrypt_AES_CBC(API_key,encrypted_text)
#     print(descryptedMsg)
    DecryptedJSON=json.loads(descryptedMsg)
#     print(DecryptedJSON)
    list1 = [(k, v) for k, v in DecryptedJSON.items()]
    output.append(list1)
print(output)


2it [00:00,  3.73it/s]

{"resAcctdtls":"8ZUJxsGdWQiTQsCTqBG6xFDC3b8aRtij1eVusc7Y8lc0dL45RplECDWGLNpLLyH3a09rmOx8y4FfIuJyHkObvHpCZiKQTbi6zSOGFrnE+3r2z+bE3j+/v3C4/j5XqZd8kzv1U2Pcrcxq9zjidGBt1rrib/zWQ0MZ5GlZnW6M68marMxq8GdK5OqjP8UfftIwaoor4Evbo211opek9jiQug=="}
{"resAcctdtls":"8ZUJxsGdWQiTQsCTqBG6xFDC3b8aRtij1eVusc7Y8lfq6mMHuC/MGiNzdCn60IQMoEW2aY4PDGsyAlSYK6+xBzU08sRgsX4by79rco/x6+HOGu52LJyU9kcufL2Y4Y31qzSFzKE+ZNd7Pi2mCNq4NxE8ttLZD2wYT/BcYk/3Qp+ErVe5Gr4H6QbgdBXoesW74YmUh8Km0i6Qw/5B00LGhQ=="}
[[('success', '1'), ('seqid', '20230102707998'), ('pan', ''), ('ifsc', ''), ('bankaccountno', ''), ('Response', {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauthorized access'})], [('success', '1'), ('seqid', '20230102921299'), ('pan', ''), ('ifsc', ''), ('bankaccountno', ''), ('Response', {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauthorized access'})]]


In [29]:
# config = ConfigParser()
# config.read('Config.ini')
# config_data=config['SERVER']
# config_data["read"]
# print(config_data["read"])
# conn = pyodbc.connect(config_data["read"])
# cursor = conn.cursor()
# cursor = conn.cursor()
# df=pd.read_sql('SELECT  * FROM [EXTL_DB].[CDSL].[Cdsl_Request_Input]',conn)
# df
# config = ConfigParser()
# config.read('er_config.ini')
# config_data=config['SERVER']
# config_data["connection"]
# conn = pyodbc.connect(config_data["connection"])
# cursor = conn.cursor()

In [30]:
df=pd.DataFrame(output)

df['status_code']=status_code
df['success'] = df[0].str[1]
df['seqid'] = df[1].str[1]
df['pan'] = df[2].str[1]
df['ifsc'] = df[3].str[1]
df['bankaccountno'] = df[4].str[1]
df['accountdtls'] = df[5].str[1]
df1=df.drop([0,1,2,3,4,5,'pan','ifsc','bankaccountno'], axis=1)
print(df1)

   status_code success           seqid  \
0          200       1  20221101440272   
1          200       1  20221101650473   
2          200       1  20221101658912   
3          200       1  20221101756647   
4          200       1  20221101873275   
5          200       1  20221101948112   
6          200       1  20230102707998   
7          200       1  20230102921299   

                                         accountdtls  
0  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  
1  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  
2  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  
3  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  
4  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  
5  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  
6  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  
7  {'ErrorId': 'CERR005', 'ErrorMessage': 'Unauth...  


In [ ]:
df2 =df1['accountdtls'].apply(lambda x: [[i['dematacctno'],i['pan'],i['ifsc'],i['bankaccountno'],i['date'],i['valuation']] for i in x]).explode().apply(pd.Series, index=['dematacctno','pan','ifsc','bankaccountno','date','valuation'])

In [ ]:
result = pd.concat([df1, df2], axis=1)
result=result.drop(['accountdtls'], axis=1)

In [ ]:
result

In [ ]:
result.rename(columns = {'date':'valuation_date','dematacctno':'Demat_Account_Number','pan':'Pan','ifsc':'Ifsc_Code'}, inplace = True)

In [ ]:
now=datetime.datetime.now()
result['Create_Date']=now
result['Create_By']='API Code'
result['Create_Job_Id']=1

result=result.fillna(0)

result['valuation_date'] =  pd.to_datetime(result['valuation_date'], format='%d%m%Y',errors='coerce')

result.replace({np.nan: None}, inplace = True)
result['Create_Date'] = pd.to_datetime(result['Create_Date'])

In [ ]:
# result['valuation_date']

In [ ]:
result.dtypes


In [ ]:
for row in tqdm(result.itertuples()):
    cursor.execute('''INSERT INTO [EXTL_DB].[CDSL].[Cdsl_Response_Output] (Api_Response_Code,Api_Success_Flag,Seqid,Demat_Account_Number,Pan,Ifsc_Code,Account_Number,Valuation_Date,Valuation,Create_Date,Create_By,Create_Job_Id) 
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?)''',row.status_code,row.success,row.seqid,row.Demat_Account_Number,row.Pan,row.Ifsc_Code,row.bankaccountno,row.valuation_date,row.valuation,row.Create_Date,row.Create_By,row.Create_Job_Id)
    cursor.execute(f"UPDATE [EXTL_DB].[CDSL].[Cdsl_Request_Input] SET Api_Request_Sent = 1 WHERE Seqid={row.seqid}")
conn.commit()

In [ ]:
#this is for updation of [EXTL_DB].[CDSL].[Cdsl_Request_Input] column Api_Request_Sent
#for row in tqdm(result.itertuples()):
# cursor.execute('''UPDATE [EXTL_DB].[CDSL].[Cdsl_Request_Input] SET Api_Request_Sent = 1''')
# conn.commit()

